# Week 3 Assignment
# Segmenting Neighborhoods in Toronto 

## First Part. Web scraping and turning HTML table into pandas dataframe

In [1]:
#import libraries

import urllib.request, urllib.parse, urllib.error
import pandas as pd
import numpy as np

from sklearn.cluster import KMeans

#!pip install folium
import folium

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import json
import requests
from pandas.io.json import json_normalize

In [2]:
#use urllib to read the web page to be scrapped
fhand = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
type(fhand)

http.client.HTTPResponse

In [3]:
#read webpage into str
data = fhand.read().decode()
type(data)

str

In [4]:
#use pandas to read the html tables, return a list of tables
postal_code = pd.read_html(data)
type(postal_code)

list

#### The table needed for the assignment is in the first position of the list

In [5]:
postal_code[0][:5]

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [6]:
toronto_borough_df = postal_code[0]
toronto_borough_df.rename(columns={"Neighbourhood":"Neighborhood"}, inplace=True)
toronto_borough_df.head(10)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [7]:
toronto_borough_df.shape

(180, 3)

#### We need to remove the rows with no Boroughs assigned

In [8]:
#find indexes with Borough == "Not assigned" and drop them from the DataFrame
drop_indexes = toronto_borough_df[toronto_borough_df.Borough == "Not assigned"].index
toronto_borough_df.drop(drop_indexes, inplace=True)
toronto_borough_df.shape

(103, 3)

#### Looked for duplicated values but there are none

In [9]:
toronto_borough_df.duplicated(subset=['Postal Code']).value_counts()

False    103
dtype: int64

#### Look for rows with Not assigned values in Neighbourhoud column. This returns an empty index array.

In [10]:
na_neighbours = toronto_borough_df[toronto_borough_df.Neighborhood == "Not assigned"].index
na_neighbours
#toronto_borough_df.iloc[na_neighbours].Neighbourhood = toronto_borough_df.iloc[na_neighbours].Borough

Int64Index([], dtype='int64')

#### Sort values and reset indexes to get the DataFrame as asked in the assignment

In [11]:
toronto_borough_df.sort_values('Postal Code', inplace=True)
toronto_borough_df.reset_index(inplace=True)
toronto_borough_df.drop('index', axis=1, inplace=True)
toronto_borough_df.head(10)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [12]:
toronto_borough_df.shape

(103, 3)

## Part 2. Dataframe with Latitude and Longitude for every Neighborhood

### Tried the geocoder package but couldn't get location data

"""!pip install geocoder
import geocoder # import geocoder

latitude = []
longitude = []

for postal_code in toronto_borough_df['Postal Code'].to_list():
    # initialize your variable to None
    lat_lng_coords = None
        # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])"""

#g = geocoder.google('{}, Toronto, Ontario'.format('M3V'))
#g

Read csv file provided in the assignment with the geospatial information for the neighborhoods in Toronto 

In [13]:
geospatial_info = pd.read_csv('https://cocl.us/Geospatial_data')

In [14]:
geospatial_info.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


Merging the Longitude and Latitude from geospatial_info with the toronto_borough_df and checking if there are any Null values

In [15]:
toronto_borough_df = toronto_borough_df.merge(geospatial_info, how='left', on='Postal Code')
print(toronto_borough_df.Latitude.isnull().value_counts())
print(toronto_borough_df.Longitude.isnull().value_counts())

False    103
Name: Latitude, dtype: int64
False    103
Name: Longitude, dtype: int64


Show the DataFrame as asked in the second part of the assignment.

In [16]:
toronto_borough_df.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


### Now we can get a map showing Toronto Postal codes Areas

In [17]:
toronto_coord = [43.6532, -79.3832]
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=toronto_coord, zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_borough_df['Latitude'], toronto_borough_df['Longitude'], 
                                           toronto_borough_df['Borough'], toronto_borough_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Third Part. Clustering neighborhood in Toronto

In [18]:
#fill Foursquare information
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

#### Use the function from the New York clustering Lab to get a maximum of 100 venues in a range of 1000m for every postal code in the table

In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
def getNearbyVenues2(names, latitudes, longitudes, radius=500, limit=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
toronto_venues = getNearbyVenues2(names=toronto_borough_df['Neighborhood'],
                                   latitudes=toronto_borough_df['Latitude'],
                                   longitudes=toronto_borough_df['Longitude'], radius=1000
                                  )

In [22]:
#toronto_venues.shape
toronto_venues.duplicated(subset=['Venue']).value_counts()
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
#toronto_venues.groupby('Neighborhood').count()
toronto_venues.head(20)

There are 328 uniques categories.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Images Salon & Spa,43.802283,-79.198565,Spa
1,"Malvern, Rouge",43.806686,-79.194353,Harvey's,43.800020,-79.198307,Restaurant
2,"Malvern, Rouge",43.806686,-79.194353,Wendy's,43.802008,-79.198080,Fast Food Restaurant
3,"Malvern, Rouge",43.806686,-79.194353,Staples Morningside,43.800285,-79.196607,Paper / Office Supplies Store
4,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
5,"Malvern, Rouge",43.806686,-79.194353,RBC Royal Bank,43.798782,-79.197090,Bank
6,"Malvern, Rouge",43.806686,-79.194353,Caribbean Wave,43.798558,-79.195777,Caribbean Restaurant
7,"Malvern, Rouge",43.806686,-79.194353,Tim Hortons,43.802000,-79.198169,Coffee Shop
8,"Malvern, Rouge",43.806686,-79.194353,Lee Valley,43.803161,-79.199681,Hobby Shop
9,"Malvern, Rouge",43.806686,-79.194353,Bus Stop: 85 & 116,43.802198,-79.199389,Bus Station


#### One hot encoding for every Venue Category retrieved in toronto_venues

In [23]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
toronto_onehot.set_index('Neighborhood', inplace=True)
toronto_onehot.reset_index(inplace=True)
#fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
#toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()
#toronto_onehot.columns=='Neighborhood'

,Neighborhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,...,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
#group by Neighborhood using mean aggregate to obtain the frequency for each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()


,Neighborhood,Accessories Store,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,...,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Agincourt,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.00000,0.022222,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,...,0.00000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.02381,0.0,0.0,0.0,0.0,...,0.02381,0.000000,0.0,0.0,0.0,0.0,0.02381,0.0,0.0,0.0


#### Using the function from the New York lab we can get the most common venues for each postal code in Toronto and will use the 10 most common to train a KNN Algorithm

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Shopping Mall,Caribbean Restaurant,Bakery,Sandwich Place,Cantonese Restaurant,Latin American Restaurant,Sri Lankan Restaurant,Breakfast Spot,Lounge
1,"Alderwood, Long Branch",Discount Store,Pizza Place,Pharmacy,Convenience Store,Coffee Shop,Skating Rink,Shopping Mall,Liquor Store,Donut Shop,Garden Center
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Convenience Store,Coffee Shop,Fried Chicken Joint,Gas Station,Mobile Phone Shop,Chinese Restaurant,Dog Run,Sandwich Place,Sushi Restaurant
3,Bayview Village,Bank,Gas Station,Grocery Store,Japanese Restaurant,Shopping Mall,Chinese Restaurant,Park,Restaurant,Café,Trail
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Fast Food Restaurant,Bank,Pizza Place,Restaurant,Sandwich Place,Juice Bar,Liquor Store,Bakery


In [27]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 1, 2, 1, 1, 1, 2])

#### Join the Cluster Labels with the neighborhoods_venues_sorted dataframe

In [29]:
#add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

neighborhoods_venues_sorted['Cluster Labels'] = neighborhoods_venues_sorted['Cluster Labels']
print(neighborhoods_venues_sorted.head())

toronto_merged = toronto_borough_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.dropna(inplace=True)
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)

toronto_merged.drop(['Postal Code', 'Borough'], inplace=True, axis=1)

toronto_merged.head() # check the last columns!

   Cluster Labels                                     Neighborhood  \
0               0                                        Agincourt   
1               0                           Alderwood, Long Branch   
2               0  Bathurst Manor, Wilson Heights, Downsview North   
3               0                                  Bayview Village   
4               1                Bedford Park, Lawrence Manor East   

  1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
0    Chinese Restaurant         Shopping Mall  Caribbean Restaurant   
1        Discount Store           Pizza Place              Pharmacy   
2                  Bank     Convenience Store           Coffee Shop   
3                  Bank           Gas Station         Grocery Store   
4    Italian Restaurant           Coffee Shop  Fast Food Restaurant   

  4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
0                Bakery        Sandwich Place  Cantonese Restaurant   
1     Conv

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Malvern, Rouge",43.806686,-79.194353,2,Coffee Shop,Trail,Fast Food Restaurant,Bakery,Auto Workshop,Gym,Restaurant,Chinese Restaurant,Supermarket,Paper / Office Supplies Store
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,4,Breakfast Spot,Park,Burger Joint,Playground,Italian Restaurant,Fast Food Restaurant,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant
2,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0,Pizza Place,Bank,Fast Food Restaurant,Coffee Shop,Park,Smoothie Shop,Food & Drink Shop,Sports Bar,Fried Chicken Joint,Supermarket
3,Woburn,43.770992,-79.216917,2,Park,Coffee Shop,Mobile Phone Shop,Indian Restaurant,Chinese Restaurant,Fast Food Restaurant,Fish & Chips Shop,Falafel Restaurant,Electronics Store,Food
4,Cedarbrae,43.773136,-79.239476,2,Coffee Shop,Bank,Gas Station,Bakery,Indian Restaurant,Intersection,Burger Joint,Fast Food Restaurant,Sporting Goods Shop,Music Store


### Visualize the resulting clusters

In [30]:
# create map
map_clusters = folium.Map(location=toronto_coord, zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Now we can check every cluster. For the maps we can see there are two outliers. They turned out to be cluster 3 where there is a Zoo and cluster 4 where the most common venue is breakfast spot

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[0] + list(range(4, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Guildwood, Morningside, West Hill",Pizza Place,Bank,Fast Food Restaurant,Coffee Shop,Park,Smoothie Shop,Food & Drink Shop,Sports Bar,Fried Chicken Joint,Supermarket
5,Scarborough Village,Ice Cream Shop,Convenience Store,Train Station,Coffee Shop,Grocery Store,Restaurant,Fast Food Restaurant,Japanese Restaurant,Pizza Place,Bowling Alley
6,"Kennedy Park, Ionview, East Birchmount Park",Chinese Restaurant,Coffee Shop,Fast Food Restaurant,Grocery Store,Pizza Place,Department Store,Burger Joint,Bank,Sandwich Place,Pharmacy
7,"Golden Mile, Clairlea, Oakridge",Intersection,Bakery,Coffee Shop,Metro Station,Park,Bus Station,Mexican Restaurant,Pizza Place,Fast Food Restaurant,Beer Store
8,"Cliffside, Cliffcrest, Scarborough Village West",Pizza Place,Ice Cream Shop,Beach,Sports Bar,Park,Restaurant,Hardware Store,Farm,Eastern European Restaurant,Electronics Store
11,"Wexford, Maryvale",Middle Eastern Restaurant,Pizza Place,Grocery Store,Burger Joint,Indian Restaurant,Korean Restaurant,Seafood Restaurant,Fish Market,Bakery,Badminton Court
12,Agincourt,Chinese Restaurant,Shopping Mall,Caribbean Restaurant,Bakery,Sandwich Place,Cantonese Restaurant,Latin American Restaurant,Sri Lankan Restaurant,Breakfast Spot,Lounge
13,"Clarks Corners, Tam O'Shanter, Sullivan",Coffee Shop,Pharmacy,Fast Food Restaurant,Bank,Sandwich Place,Bakery,Noodle House,Thai Restaurant,Discount Store,Gas Station
14,"Milliken, Agincourt North, Steeles East, L'Amo...",Chinese Restaurant,Korean Restaurant,Pizza Place,Bakery,Park,Hobby Shop,Gym,Shop & Service,Shopping Mall,Malay Restaurant
15,"Steeles West, L'Amoreaux West",Chinese Restaurant,Pizza Place,Bank,Bakery,Coffee Shop,Fast Food Restaurant,Intersection,Sandwich Place,Noodle House,Other Great Outdoors


In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[0] + list(range(4, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,"Birch Cliff, Cliffside West",Convenience Store,College Stadium,Thai Restaurant,Park,Diner,Gym,Café,Gym Pool,Restaurant,Skating Rink
21,"Willowdale, Newtonbrook",Korean Restaurant,Café,Pizza Place,Bus Station,Middle Eastern Restaurant,Coffee Shop,Park,Grocery Store,Shopping Mall,Fried Chicken Joint
22,"Willowdale, Willowdale East",Coffee Shop,Bubble Tea Shop,Ramen Restaurant,Pizza Place,Japanese Restaurant,Korean Restaurant,Sandwich Place,Sushi Restaurant,Restaurant,Bank
26,Don Mills,Restaurant,Japanese Restaurant,Coffee Shop,Gym,Bank,Pizza Place,Supermarket,Burger Joint,Asian Restaurant,Beer Store
27,Don Mills,Restaurant,Japanese Restaurant,Coffee Shop,Gym,Bank,Pizza Place,Supermarket,Burger Joint,Asian Restaurant,Beer Store
37,The Beaches,Coffee Shop,Pub,Pizza Place,Beach,Japanese Restaurant,Breakfast Spot,Caribbean Restaurant,Burger Joint,Bakery,Indian Restaurant
40,"East Toronto, Broadview North (Old East York)",Coffee Shop,Café,Greek Restaurant,Ethiopian Restaurant,Pharmacy,Pizza Place,Beer Bar,Fast Food Restaurant,Flower Shop,Gastropub
41,"The Danforth West, Riverdale",Greek Restaurant,Café,Coffee Shop,Pub,Ice Cream Shop,Fast Food Restaurant,Italian Restaurant,Bank,Spa,Discount Store
42,"India Bazaar, The Beaches West",Indian Restaurant,Coffee Shop,Beach,Café,Harbor / Marina,Sandwich Place,Brewery,Bakery,Fast Food Restaurant,Pizza Place
43,Studio District,Coffee Shop,Bar,Café,Diner,Brewery,Bakery,American Restaurant,Vietnamese Restaurant,Italian Restaurant,French Restaurant


In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[0] + list(range(4, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Malvern, Rouge",Coffee Shop,Trail,Fast Food Restaurant,Bakery,Auto Workshop,Gym,Restaurant,Chinese Restaurant,Supermarket,Paper / Office Supplies Store
3,Woburn,Park,Coffee Shop,Mobile Phone Shop,Indian Restaurant,Chinese Restaurant,Fast Food Restaurant,Fish & Chips Shop,Falafel Restaurant,Electronics Store,Food
4,Cedarbrae,Coffee Shop,Bank,Gas Station,Bakery,Indian Restaurant,Intersection,Burger Joint,Fast Food Restaurant,Sporting Goods Shop,Music Store
10,"Dorset Park, Wexford Heights, Scarborough Town...",Furniture / Home Store,Coffee Shop,Pharmacy,Restaurant,Chinese Restaurant,Bakery,Indian Restaurant,Fast Food Restaurant,Accessories Store,Burger Joint
18,"Fairview, Henry Farm, Oriole",Coffee Shop,Clothing Store,Bank,Japanese Restaurant,Bakery,Restaurant,Juice Bar,Sandwich Place,Fast Food Restaurant,Caribbean Restaurant
23,York Mills West,Park,Restaurant,Coffee Shop,Dentist's Office,Pet Store,Gas Station,French Restaurant,Golf Course,Grocery Store,Bowling Alley
29,"Northwood Park, York University",Furniture / Home Store,Pizza Place,Coffee Shop,Restaurant,Bank,Japanese Restaurant,Market,Sandwich Place,Massage Studio,Fast Food Restaurant
34,Victoria Village,Coffee Shop,Intersection,Grocery Store,Hockey Arena,Lounge,Sporting Goods Shop,Portuguese Restaurant,Golf Course,French Restaurant,Men's Store
36,Woodbine Heights,Coffee Shop,Pizza Place,Sandwich Place,Skating Rink,Park,Thai Restaurant,Farmers Market,Beer Store,Liquor Store,Café
38,Leaside,Coffee Shop,Sporting Goods Shop,Furniture / Home Store,Grocery Store,Sandwich Place,Burger Joint,Restaurant,Brewery,Department Store,Electronics Store


In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[0] + list(range(4, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,"York Mills, Silver Hills",Park,Pool,Zoo,Farmers Market,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant


In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[0] + list(range(4, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Rouge Hill, Port Union, Highland Creek",Breakfast Spot,Park,Burger Joint,Playground,Italian Restaurant,Fast Food Restaurant,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant
